In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('data_motores.csv')
pd.set_option('display.max_columns', None)

In [3]:
df

,Planta,Distancia,Motor,Potencia (HP),Eficiencia (%),Factor de carga maximo,Factor de potencia,Potencia Entrada (W),Corriente a 220V (A),Corriente a 380V (A)
0,Planta 1,200,1,30,70,80,0.74,"31971,43",113.382757,65.642649
1,Planta 2,300,1,40,79,80,0.82,"37772,15",120.885599,69.986399
2,Planta 3,350,1,60,85,60,0.78,"52658,82",177.171247,102.572827
3,Planta 4,300,1,30,82,60,0.78,"27292,68",91.826561,53.162746
4,Planta 5,500,1,100,91,60,0.81,"81978,02",265.600634,153.768788
5,Planta 1,200,2,50,85,80,0.75,"43882,35",153.548414,88.896450
6,Planta 2,300,2,125,88,80,0.83,"105965,91",335.046255,193.974148
7,Planta 3,350,2,150,90,80,0.79,"124333,33",413.025797,239.120198
8,Planta 4,300,2,40,82,60,0.79,"36390,24",120.885599,69.986399
9,Planta 5,500,2,250,92,60,0.82,"202717,39",648.774614,375.606356


In [4]:
column_names = df.columns
print(column_names)


Index(['Planta', 'Distancia', 'Motor', 'Potencia (HP)', 'Eficiencia (%)',
       'Factor de carga maximo', 'Factor de potencia', 'Potencia Entrada (W)',
       'Corriente a 220V (A)', 'Corriente a 380V (A)'],
      dtype='object')


In [5]:
df1 = df.copy()

In [6]:
df1['Potencia Entrada (W)'] = (df1['Potencia (HP)'] * 746 * (df1['Factor de carga maximo']/100)) / (df1['Eficiencia (%)'] / 100)

In [7]:
df1['Corriente a 220V (A)'] = df1['Potencia Entrada (W)'] / (np.sqrt(3) * 220 * df1['Factor de potencia'])
df1['Corriente a 380V (A)'] = df1['Potencia Entrada (W)'] / (np.sqrt(3) * 380 * df1['Factor de potencia'])

In [8]:
df1

,Planta,Distancia,Motor,Potencia (HP),Eficiencia (%),Factor de carga maximo,Factor de potencia,Potencia Entrada (W),Corriente a 220V (A),Corriente a 380V (A)
0,Planta 1,200,1,30,70,80,0.74,25577.142857,90.706206,52.514119
1,Planta 2,300,1,40,79,80,0.82,30217.721519,96.708479,55.989120
2,Planta 3,350,1,60,85,60,0.78,31595.294118,106.302748,61.543696
3,Planta 4,300,1,30,82,60,0.78,16375.609756,55.095936,31.897647
4,Planta 5,500,1,100,91,60,0.81,49186.813187,159.360381,92.261273
5,Planta 1,200,2,50,85,80,0.75,35105.882353,122.838731,71.117160
6,Planta 2,300,2,125,88,80,0.83,84772.727273,268.037004,155.179318
7,Planta 3,350,2,150,90,80,0.79,99466.666667,330.420637,191.296158
8,Planta 4,300,2,40,82,60,0.79,21834.146341,72.531359,41.991840
9,Planta 5,500,2,250,92,60,0.82,121630.434783,389.264769,225.363813


In [9]:
df1['Corriente a 220V (A) +25%'] = df1['Corriente a 220V (A)']*1.25
df1['Corriente a 380V (A) +25%'] = df1['Corriente a 380V (A)']*1.25

In [10]:
df1['Velocidad síncrona (RPM)'] = 3600

In [11]:
df1

,Planta,Distancia,Motor,Potencia (HP),Eficiencia (%),Factor de carga maximo,Factor de potencia,Potencia Entrada (W),Corriente a 220V (A),Corriente a 380V (A),Corriente a 220V (A) +25%,Corriente a 380V (A) +25%,Velocidad síncrona (RPM)
0,Planta 1,200,1,30,70,80,0.74,25577.142857,90.706206,52.514119,113.382757,65.642649,3600
1,Planta 2,300,1,40,79,80,0.82,30217.721519,96.708479,55.989120,120.885599,69.986399,3600
2,Planta 3,350,1,60,85,60,0.78,31595.294118,106.302748,61.543696,132.878435,76.929620,3600
3,Planta 4,300,1,30,82,60,0.78,16375.609756,55.095936,31.897647,68.869921,39.872059,3600
4,Planta 5,500,1,100,91,60,0.81,49186.813187,159.360381,92.261273,199.200476,115.326591,3600
5,Planta 1,200,2,50,85,80,0.75,35105.882353,122.838731,71.117160,153.548414,88.896450,3600
6,Planta 2,300,2,125,88,80,0.83,84772.727273,268.037004,155.179318,335.046255,193.974148,3600
7,Planta 3,350,2,150,90,80,0.79,99466.666667,330.420637,191.296158,413.025797,239.120198,3600
8,Planta 4,300,2,40,82,60,0.79,21834.146341,72.531359,41.991840,90.664199,52.489800,3600
9,Planta 5,500,2,250,92,60,0.82,121630.434783,389.264769,225.363813,486.580961,281.704767,3600


In [12]:
catalogo = pd.read_csv('catalogo_motores.csv')

In [13]:
catalogo['Potencia HP'] = catalogo['Potencia HP'].astype(int)

In [14]:
catalogo

,Unnamed: 0,No. de Depósito,Referencia,Potencia KW,Potencia HP,Tamaño constructivo,Velocidad rpm,Torque Nm,Corriente nominal 220V DD A,Corriente nominal 380V YY A,Corriente nominal 440V D A,Eficiencia %,Peso neto (IM B3) kg,Precio de lista USD
0,0,100664611,1LE0143-0DA26-4AA4-Z,0.75,1,80M,3450,2.1,3.15,1.82,1.57,77.0,15.0,270.0
1,1,100664612,1LE0143-0DA36-4AA4-Z,1.10,1,80M,3480,3.0,4.3,2.50,2.15,84.0,18.0,300.4
2,2,100664613,1LE0143-0DA86-4AA4-Z,1.50,2,80M,3465,4.1,5.7,3.25,2.85,85.5,18.5,373.8
3,3,100664614,1LE0143-0EA46-4AA4-Z,2.20,3,90L,3500,6.0,7.8,4.50,3.90,86.5,29.0,453.9
4,4,100664615,1LE0143-0EA86-4AA4-Z,3.00,4,90L,3515,8.2,10.7,6.20,5.30,88.5,29.0,521.4
5,5,100664616,1LE0143-1AA86-4AA4-Z,3.70,5,100L,3465,10.2,12.6,7.30,6.30,88.5,45.0,613.7
6,6,100664618,1LE0143-1BA86-4AA4-Z,5.50,7,112M,3520,14.9,18.6,10.80,9.30,89.5,47.0,818.6
7,7,100664619,1LE0143-1CA16-4AA4-Z,7.50,10,132S,3535,20.3,24.0,14.00,12.10,90.2,76.0,1016.9
8,8,100664650,1LE0143-1CA86-4AA4-Z,11.00,15,132M,3540,29.7,36.0,21.00,17.90,91.0,79.0,1365.8
9,9,100664651,1LE0143-1DA36-4AA4-Z,15.00,20,160M,3540,40.5,52.0,30.00,26.00,91.0,103.0,1750.1


In [15]:
df2 = pd.merge(df1, catalogo[['Potencia HP', 'Velocidad rpm']], left_on='Potencia (HP)', right_on='Potencia HP', how='left')

In [16]:
df2.drop(columns=['Potencia HP'], inplace=True)
df2 = df2.rename(columns={'Velocidad rpm': 'Velocidad real (RPM)'})

In [17]:
df2['Deslizamiento (RPM)'] = df2['Velocidad síncrona (RPM)'] - df2['Velocidad real (RPM)'] 
df2['Deslizamiento (%)'] = ((df2['Velocidad síncrona (RPM)'] - df2['Velocidad real (RPM)']) / df2['Velocidad síncrona (RPM)'])*100

In [18]:
df2['Torque (N.m)'] = (df2['Potencia (HP)'] * 746 * (df1['Factor de carga maximo']/100)) / ((df2['Velocidad real (RPM)'] * 2 * np.pi)/60)

In [19]:
df2['theta (°)'] = np.arccos(df2['Factor de potencia'])

In [20]:
df2['Real Corriente 220V'] = df2['Corriente a 220V (A) +25%'] * np.cos(df2['theta (°)']) 
df2['Imaginaria Corriente 220V'] = df2['Corriente a 220V (A) +25%'] * np.sin(df2['theta (°)'])

df2['Real Corriente 380V'] = df2['Corriente a 380V (A) +25%'] * np.cos(df2['theta (°)']) 
df2['Imaginaria Corriente 380V'] = df2['Corriente a 380V (A) +25%'] * np.sin(df2['theta (°)'])

In [21]:
df2

,Planta,Distancia,Motor,Potencia (HP),Eficiencia (%),Factor de carga maximo,Factor de potencia,Potencia Entrada (W),Corriente a 220V (A),Corriente a 380V (A),Corriente a 220V (A) +25%,Corriente a 380V (A) +25%,Velocidad síncrona (RPM),Velocidad real (RPM),Deslizamiento (RPM),Deslizamiento (%),Torque (N.m),theta (°),Real Corriente 220V,Imaginaria Corriente 220V,Real Corriente 380V,Imaginaria Corriente 380V
0,Planta 1,200,1,30,70,80,0.74,25577.142857,90.706206,52.514119,113.382757,65.642649,3600,3540,60,1.666667,48.296781,0.737726,83.903240,76.262021,48.575560,44.151697
1,Planta 2,300,1,40,79,80,0.82,30217.721519,96.708479,55.989120,120.885599,69.986399,3600,3565,35,0.972222,63.944126,0.609385,99.126191,69.190507,57.388848,40.057662
2,Planta 3,350,1,60,85,60,0.78,31595.294118,106.302748,61.543696,132.878435,76.929620,3600,3570,30,0.833333,71.836389,0.676131,103.645179,83.152602,60.005104,48.140980
3,Planta 4,300,1,30,82,60,0.78,16375.609756,55.095936,31.897647,68.869921,39.872059,3600,3540,60,1.666667,36.222586,0.676131,53.718538,43.097385,31.100206,24.951118
4,Planta 5,500,1,100,91,60,0.81,49186.813187,159.360381,92.261273,199.200476,115.326591,3600,3570,30,0.833333,119.727315,0.626644,161.352385,116.817110,93.414539,67.630959
5,Planta 1,200,2,50,85,80,0.75,35105.882353,122.838731,71.117160,153.548414,88.896450,3600,3570,30,0.833333,79.818210,0.722734,115.161310,101.562729,66.672338,58.799475
6,Planta 2,300,2,125,88,80,0.83,84772.727273,268.037004,155.179318,335.046255,193.974148,3600,3580,20,0.555556,198.988136,0.591689,278.088392,186.876535,160.998542,108.191678
7,Planta 3,350,2,150,90,80,0.79,99466.666667,330.420637,191.296158,413.025797,239.120198,3600,3580,20,0.555556,238.785763,0.659987,326.290379,253.228942,188.904956,146.606230
8,Planta 4,300,2,40,82,60,0.79,21834.146341,72.531359,41.991840,90.664199,52.489800,3600,3565,35,0.972222,47.958094,0.659987,71.624717,55.586841,41.466942,32.181855
9,Planta 5,500,2,250,92,60,0.82,121630.434783,389.264769,225.363813,486.580961,281.704767,3600,3582,18,0.500000,298.315547,0.609385,398.996388,278.501192,230.997909,161.237532


# PARA 220V

Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 1

In [22]:
Corriente_Planta_1_220V = np.sqrt((df2[df2['Planta'] == 'Planta 1']['Real Corriente 220V'].sum())**2 + (df2[df2['Planta'] == 'Planta 1']['Imaginaria Corriente 220V'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 1: {Corriente_Planta_1_220V} A')

Magnitud de corriente a 220V para los motores de la planta 1: 740.8613845652662 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 2

In [23]:
Corriente_Planta_2_220V = np.sqrt((df2[df2['Planta'] == 'Planta 2']['Real Corriente 220V'].sum())**2 + (df2[df2['Planta'] == 'Planta 2']['Imaginaria Corriente 220V'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 2: {Corriente_Planta_2_220V} A')

Magnitud de corriente a 220V para los motores de la planta 2: 1055.846103440497 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 3

In [24]:
Corriente_Planta_3_220V = np.sqrt((df2[df2['Planta'] == 'Planta 3']['Real Corriente 220V'].sum())**2 + (df2[df2['Planta'] == 'Planta 3']['Imaginaria Corriente 220V'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 3: {Corriente_Planta_3_220V} A')

Magnitud de corriente a 220V para los motores de la planta 3: 1184.1293915625126 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 4

In [25]:
Corriente_Planta_4_220V = np.sqrt((df2[df2['Planta'] == 'Planta 4']['Real Corriente 220V'].sum())**2 + (df2[df2['Planta'] == 'Planta 4']['Imaginaria Corriente 220V'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 4: {Corriente_Planta_4_220V} A')

Magnitud de corriente a 220V para los motores de la planta 4: 774.7133890486778 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 5

In [26]:
Corriente_Planta_5_220V = np.sqrt((df2[df2['Planta'] == 'Planta 5']['Real Corriente 220V'].sum())**2 + (df2[df2['Planta'] == 'Planta 5']['Imaginaria Corriente 220V'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 5: {Corriente_Planta_5_220V} A')

Magnitud de corriente a 220V para los motores de la planta 5: 1396.9715056603966 A


# PARA 380V

Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 1

In [27]:
Corriente_Planta_1_380V = np.sqrt((df2[df2['Planta'] == 'Planta 1']['Real Corriente 380V'].sum())**2 + (df2[df2['Planta'] == 'Planta 1']['Imaginaria Corriente 380V'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 1: {Corriente_Planta_1_380V} A')

Magnitud de corriente a 380V para los motores de la planta 1: 428.91974895883834 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 2

In [28]:
Corriente_Planta_2_380V = np.sqrt((df2[df2['Planta'] == 'Planta 2']['Real Corriente 380V'].sum())**2 + (df2[df2['Planta'] == 'Planta 2']['Imaginaria Corriente 380V'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 2: {Corriente_Planta_2_380V} A')

Magnitud de corriente a 380V para los motores de la planta 2: 611.2793230444981 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 3

In [29]:
Corriente_Planta_3_380V = np.sqrt((df2[df2['Planta'] == 'Planta 3']['Real Corriente 380V'].sum())**2 + (df2[df2['Planta'] == 'Planta 3']['Imaginaria Corriente 380V'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 3: {Corriente_Planta_3_380V} A')

Magnitud de corriente a 380V para los motores de la planta 3: 685.5485951151388 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 4

In [30]:
Corriente_Planta_4_380V = np.sqrt((df2[df2['Planta'] == 'Planta 4']['Real Corriente 380V'].sum())**2 + (df2[df2['Planta'] == 'Planta 4']['Imaginaria Corriente 380V'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 4: {Corriente_Planta_4_380V} A')

Magnitud de corriente a 380V para los motores de la planta 4: 448.51827787028714 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 5

In [31]:
Corriente_Planta_5_380V = np.sqrt((df2[df2['Planta'] == 'Planta 5']['Real Corriente 380V'].sum())**2 + (df2[df2['Planta'] == 'Planta 5']['Imaginaria Corriente 380V'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 5: {Corriente_Planta_5_380V} A')

Magnitud de corriente a 380V para los motores de la planta 5: 808.7729769612822 A
